# Query with GraphQL mongodb

## Create User Document

In [21]:
from mongoengine import Document
from mongoengine.fields import StringField

class UserModel(Document):
    meta = {'collection': 'Users'}
    _id = StringField()
    id = StringField()
    name = StringField()
    lastname = StringField()
    userIcon = StringField()

## Query User Document

In [69]:
import graphene

from graphene_mongo import MongoengineObjectType
from mongoengine import connect

connect(db="portfolio-next",host="127.0.0.1:32768")

class User(MongoengineObjectType):
    class Meta:
        model = UserModel

class Query(graphene.ObjectType):
    users = graphene.List(User)
    use = graphene.Field(User, name=graphene.String())
    
    def resolve_users(self, info):
    	return list(UserModel.objects.all())

    def resolve_use(self, info, name):
        return UserModel.objects.get(name=name)

schema = graphene.Schema(query=Query)

query = '''
    query {
        use(name: "Bon") {    
        id, name, userIcon
        }
    }
'''
result = await schema.execute_async(query)
result.data['use']

{'id': 'b349204d-5d58-3b48-bfb6-fb7fbd7ae103',
 'name': 'Bon',
 'userIcon': 'https://images.unsplash.com/photo-1685972215665-80580c58e4ee?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D&auto=format&fit=crop&w=2097&q=80'}

ExecutionResult(data={'use': {'id': 'b349204d-5d58-3b48-bfb6-fb7fbd7ae103', 'name': 'Bon', 'userIcon': 'https://images.unsplash.com/photo-1685972215665-80580c58e4ee?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D&auto=format&fit=crop&w=2097&q=80'}}, errors=None)